# Interacting with Campaigns <a class="anchor" id="top"></a>

In this notebook, you will deploy and interact with campaigns in Amazon Personalize.

1. [Introduction](#intro)
1. [Create campaigns](#create)
1. [Interact with campaigns](#interact)
1. [Batch recommendations](#batch)
1. [Wrap up](#wrapup)

## Introduction <a class="anchor" id="intro"></a>
[Back to top](#top)

At this point, you should have several solutions and at least one solution version for each. Once a solution version is created, it is possible to get recommendations from them, and to get a feel for their overall behavior.

This notebook starts off by deploying each of the solution versions from the previous notebook into individual campaigns. Once they are active, there are resources for querying the recommendations, and helper functions to digest the output into something more human-readable. 

As you with your customer on Amazon Personalize, you can modify the helper functions to fit the structure of their data input files to keep the additional rendering working.

To get started, once again, we need to import libraries, load values from previous notebooks, and load the SDK.

In [85]:
import time
from time import sleep
import json
from datetime import datetime
import uuid
import random

import boto3
import pandas as pd

In [86]:
%store -r

In [87]:
personalize = boto3.client('personalize')
personalize_runtime = boto3.client('personalize-runtime')

# Establish a connection to Personalize's event streaming
personalize_events = boto3.client(service_name='personalize-events')

## Interact with campaigns <a class="anchor" id="interact"></a>
[Back to top](#top)

Now that all campaigns are deployed and active, we can start to get recommendations via an API call. Each of the campaigns is based on a different recipe, which behave in slightly different ways because they serve different use cases. We will cover each campaign in a different order than used in previous notebooks, in order to deal with the possible complexities in ascending order (i.e. simplest first).

First, let's create a supporting function to help make sense of the results returned by a Personalize campaign. Personalize returns only an `item_id`. This is great for keeping data compact, but it means you need to query a database or lookup table to get a human-readable result for the notebooks. We will create a helper function to return a human-readable result from the LastFM dataset.

Start by loading in the dataset which we can use for our lookup table.

In [88]:
# Create a dataframe for the items by reading in the correct source CSV
items_df = pd.read_csv(dataset_dir + '/movies.csv', sep=',', usecols=[0,1], encoding='latin-1', dtype={'movieId': "object", 'title': "str"},index_col=0)

# Render some sample data
items_df.head(5)

,title
movieId,
1,Toy Story (1995)
2,Jumanji (1995)
3,Grumpier Old Men (1995)
4,Waiting to Exhale (1995)
5,Father of the Bride Part II (1995)


By defining the ID column as the index column it is trivial to return an artist by just querying the ID.

In [89]:
movie_id_example = 589
title = items_df.loc[movie_id_example]['title']
print(title)

Terminator 2: Judgment Day (1991)


That isn't terrible, but it would get messy to repeat this everywhere in our code, so the function below will clean that up.

In [90]:
def get_movie_by_id(movie_id, movie_df=items_df):
    """
    This takes in an artist_id from Personalize so it will be a string,
    converts it to an int, and then does a lookup in a default or specified
    dataframe.
    
    A really broad try/except clause was added in case anything goes wrong.
    
    Feel free to add more debugging or filtering here to improve results if
    you hit an error.
    """
    try:
        return movie_df.loc[int(movie_id)]['title']
    except:
        return "Error obtaining title"

Now let's test a few simple values to check our error catching.

In [91]:
# A known good id (The Princess Bride)
print(get_movie_by_id(movie_id="1197"))
# A bad type of value
print(get_movie_by_id(movie_id="987.9393939"))
# Really bad values
print(get_movie_by_id(movie_id="Steve"))

Princess Bride, The (1987)
Error obtaining title
Error obtaining title


Great! Now we have a way of rendering results. 

### SIMS

SIMS requires just an item as input, and it will return items which users interact with in similar ways to their interaction with the input item. In this particular case the item is a movie. 

The cells below will handle getting recommendations from SIMS and rendering the results. Let's see what the recommendations are for the first item we looked at earlier in this notebook (Terminator 2: Judgment Day).

In [92]:
get_recommendations_response = personalize_runtime.get_recommendations(
    campaignArn = sims_campaign_arn,
    itemId = str(589),
)

In [93]:
item_list = get_recommendations_response['itemList']
for item in item_list:
    print(get_movie_by_id(movie_id=item['itemId']))

Jurassic Park (1993)
Fugitive, The (1993)
Speed (1994)
Terminator, The (1984)
True Lies (1994)
Braveheart (1995)
Die Hard: With a Vengeance (1995)
Batman (1989)
In the Line of Fire (1993)
Stargate (1994)
Clear and Present Danger (1994)
Apollo 13 (1995)
Forrest Gump (1994)
Firm, The (1993)
Star Trek: Generations (1994)
Die Hard (1988)
Ghost (1990)
Mask, The (1994)
Dances with Wolves (1990)
GoldenEye (1995)
Demolition Man (1993)
Silence of the Lambs, The (1991)
Cliffhanger (1993)
Aliens (1986)
Mrs. Doubtfire (1993)


Congrats, this is your first list of recommendations! This list is fine, but it would be better to see the recommendations for our sample collection of artists render in a nice dataframe. Again, let's create a helper function to achieve this.

In [94]:
# Update DF rendering
pd.set_option('display.max_rows', 30)

def get_new_recommendations_df(recommendations_df, movie_ID):
    # Get the movie name
    movie_name = get_movie_by_id(movie_ID)
    # Get the recommendations
    get_recommendations_response = personalize_runtime.get_recommendations(
        campaignArn = sims_campaign_arn,
        itemId = str(movie_ID),
    )
    # Build a new dataframe of recommendations
    item_list = get_recommendations_response['itemList']
    recommendation_list = []
    for item in item_list:
        movie = get_movie_by_id(item['itemId'])
        recommendation_list.append(movie)
    new_rec_DF = pd.DataFrame(recommendation_list, columns = [movie_name])
    # Add this dataframe to the old one
    recommendations_df = pd.concat([recommendations_df, new_rec_DF], axis=1)
    return recommendations_df

Now, let's test the helper function with several different movies. Let's sample some data from our dataset to test our SIMS campaign. Grab 5 random movies from our dataframe.

Note: We are going to show similar titles, so you may want to re-run the sample until you recognize some of the movies listed

In [99]:
samples = items_df.sample(5)
samples

,title
movieId,
189271,My Sweet Audrina (2016)
172059,The Two Worlds of Jennie Logan (1979)
101823,Poor White Trash (2000)
157230,Disney's Very Merry Christmas Sing Along Songs...
131094,Rudolph the Red-Nosed Reindeer: The Movie (1998)


In [100]:
sims_recommendations_df = pd.DataFrame()
movies = samples.index.tolist()

for movie in movies:
    sims_recommendations_df = get_new_recommendations_df(sims_recommendations_df, movie)

sims_recommendations_df

,My Sweet Audrina (2016),The Two Worlds of Jennie Logan (1979),Poor White Trash (2000),Disney's Very Merry Christmas Sing Along Songs (1988),Rudolph the Red-Nosed Reindeer: The Movie (1998)
0,"Shawshank Redemption, The (1994)","Shawshank Redemption, The (1994)",Spring Breakers (2013),"Strawberry Shortcake: Berry, Merry Christmas (...",Rio 2 (2014)
1,Forrest Gump (1994),Forrest Gump (1994),NaN,Stellaluna (2004),Monster House (2006)
2,Pulp Fiction (1994),Pulp Fiction (1994),NaN,"Bah, Humduck!: A Looney Tunes Christmas (2006)",LEGO Hero Factory: Savage Planet (2011)
3,"Silence of the Lambs, The (1991)","Silence of the Lambs, The (1991)",NaN,Pooh's Heffalump Halloween Movie (2005),Hunter Ã Hunter: Phantom Rouge (2013)
4,"Matrix, The (1999)","Matrix, The (1999)",NaN,Scary Godmother The Halloween Spooktakular (2003),Hunter Ã Hunter: The Last Mission (2013)
5,Star Wars: Episode IV - A New Hope (1977),Star Wars: Episode IV - A New Hope (1977),NaN,The Ghost of Lord Farquaad (2003),"Turtle's Tale: Sammy's Adventures, A (2010)"
6,Schindler's List (1993),Schindler's List (1993),NaN,Charlie Brown's Christmas Tales (2002),Sword Art Online Extra Edition (2013)
7,Jurassic Park (1993),Jurassic Park (1993),NaN,"I Want a Dog for Christmas, Charlie Brown (2003)",First Squad: The Moment of Truth (2009)
8,Braveheart (1995),Braveheart (1995),NaN,How to Hook Up Your Home Theater (2007),Next Avengers: Heroes of Tomorrow (2008)
9,Fight Club (1999),Fight Club (1999),NaN,Clifford's Really Big Movie (2004),Gurren Lagann: The Lights in the Sky are Stars...


You may notice that a lot of the items look the same, hopefully not all of them do (this is more likely with a smaller # of interactions). This shows that the evaluation metrics should not be the only thing you rely on when evaluating your solution version. So when this happens, what can you do to improve the results?

This is a good time to think about the hyperparameters of the Personalize recipes. The SIMS recipe has a `popularity_discount_factor` hyperparameter (see [documentation](https://docs.aws.amazon.com/personalize/latest/dg/native-recipe-sims.html)). Leveraging this hyperparameter allows you to control the nuance you see in the results. This parameter and its behavior will be unique to every dataset you encounter, and depends on the goals of the business. You can iterate on the value of this hyperparameter until you are satisfied with the results, or you can start by leveraging Personalize's hyperparameter optimization (HPO) feature. For more information on hyperparameters and HPO tuning, see the [documentation](https://docs.aws.amazon.com/personalize/latest/dg/customizing-solution-config-hpo.html).

### User Personalization

HRNN is one of the more advanced algorithms provided by Amazon Personalize. It supports personalization of the items for a specific user based on their past behavior and can intake real time events in order to alter recommendations for a user without retraining. 

Since HRNN relies on having a sampling of users, let's load the data we need for that and select 3 random users. Since Movielens does not include user data, we will select 3 random numbers from the range of user id's in the dataset.

In [101]:
if not USE_FULL_MOVIELENS:
    users = random.sample(range(1, 600), 3)
else:
    users = random.sample(range(1, 162000), 3)
users

[93562, 61007, 43392]

Now we render the recommendations for our 3 random users from above. After that, we will explore real-time interactions before moving on to Personalized Ranking.

Again, we create a helper function to render the results in a nice dataframe.

#### API call results

In [102]:
# Update DF rendering
pd.set_option('display.max_rows', 30)

def get_new_recommendations_df_users(recommendations_df, user_id):
    # Get the movie name
    #movie_name = get_movie_by_id(artist_ID)
    # Get the recommendations
    get_recommendations_response = personalize_runtime.get_recommendations(
        campaignArn = userpersonalization_campaign_arn,
        userId = str(user_id),
    )
    # Build a new dataframe of recommendations
    item_list = get_recommendations_response['itemList']
    recommendation_list = []
    for item in item_list:
        movie = get_movie_by_id(item['itemId'])
        recommendation_list.append(movie)
    #print(recommendation_list)
    new_rec_DF = pd.DataFrame(recommendation_list, columns = [user_id])
    # Add this dataframe to the old one
    recommendations_df = pd.concat([recommendations_df, new_rec_DF], axis=1)
    return recommendations_df

In [104]:
recommendations_df_users = pd.DataFrame()
#users = users_df.sample(3).index.tolist()

for user in users:
    recommendations_df_users = get_new_recommendations_df_users(recommendations_df_users, user)

recommendations_df_users

,93562,61007,43392
0,Big Fish (2003),Reservoir Dogs (1992),Chinatown (1974)
1,Ice Age (2002),Office Space (1999),"Sting, The (1973)"
2,Ratatouille (2007),"Big Lebowski, The (1998)",Citizen Kane (1941)
3,Slumdog Millionaire (2008),Snatch (2000),Rear Window (1954)
4,Harry Potter and the Order of the Phoenix (2007),Full Metal Jacket (1987),North by Northwest (1959)
5,District 9 (2009),Trainspotting (1996),"Graduate, The (1967)"
6,"Prestige, The (2006)",Monty Python's Life of Brian (1979),Taxi Driver (1976)
7,"Bourne Supremacy, The (2004)","Lock, Stock & Two Smoking Barrels (1998)",L.A. Confidential (1997)
8,Shaun of the Dead (2004),"Clockwork Orange, A (1971)",Goodfellas (1990)
9,Harry Potter and the Prisoner of Azkaban (2004),Hot Fuzz (2007),Vertigo (1958)


Here we clearly see that the recommendations for each user are different. If you were to need a cache for these results, you could start by running the API calls through all your users and store the results, or you could use a batch export, which will be covered later in this notebook.

Now lets apply item filters to see recommendations for one of these users within a genre


In [105]:
def get_new_recommendations_df_by_filter(recommendations_df, user_id, filter_arn):
    # Get the movie name
    #movie_name = get_movie_by_id(artist_ID)
    # Get the recommendations
    get_recommendations_response = personalize_runtime.get_recommendations(
        campaignArn = userpersonalization_campaign_arn,
        userId = str(user_id),
        filterArn = filter_arn
    )
    # Build a new dataframe of recommendations
    item_list = get_recommendations_response['itemList']
    recommendation_list = []
    for item in item_list:
        movie = get_movie_by_id(item['itemId'])
        recommendation_list.append(movie)
    #print(recommendation_list)
    filter_name = filter_arn.split('/')[1]
    new_rec_DF = pd.DataFrame(recommendation_list, columns = [filter_name])
    # Add this dataframe to the old one
    recommendations_df = pd.concat([recommendations_df, new_rec_DF], axis=1)
    return recommendations_df

In [124]:
user

43392

You can see the recommendations for movies within a given genre. Within a VOD application you could create Shelves (also known as rails or carosels) easily by using these filters. Depending on the information you have about your items, You could also filter on additional information such as keyword, year/decade etc.

In [107]:
recommendations_df_shelves = pd.DataFrame()
for filter_arn in meta_filter_arns:
    recommendations_df_shelves = get_new_recommendations_df_by_filter(recommendations_df_shelves, user, filter_arn)
for filter_arn in decade_filter_arns:
    recommendations_df_shelves = get_new_recommendations_df_by_filter(recommendations_df_shelves, user, filter_arn)

recommendations_df_shelves

,Thriller,Drama,Children,Documentary,Western,War,Mystery,1970s
0,Chinatown (1974),Citizen Kane (1941),"Wizard of Oz, The (1939)",Fog of War: Eleven Lessons from the Life of Ro...,"Good, the Bad and the Ugly, The (Buono, il bru...",Dr. Strangelove or: How I Learned to Stop Worr...,Chinatown (1974),Chinatown (1974)
1,Rear Window (1954),"Graduate, The (1967)",It's a Wonderful Life (1946),Hoop Dreams (1994),Unforgiven (1992),Apocalypse Now (1979),Citizen Kane (1941),"Sting, The (1973)"
2,North by Northwest (1959),Taxi Driver (1976),Wallace & Gromit: The Wrong Trousers (1993),Exit Through the Gift Shop (2010),Butch Cassidy and the Sundance Kid (1969),Life Is Beautiful (La Vita Ã¨ bella) (1997),Rear Window (1954),Taxi Driver (1976)
3,Taxi Driver (1976),Goodfellas (1990),Wallace & Gromit: A Close Shave (1995),Bowling for Columbine (2002),Django Unchained (2012),"Manchurian Candidate, The (1962)",North by Northwest (1959),Apocalypse Now (1979)
4,L.A. Confidential (1997),Vertigo (1958),"Christmas Story, A (1983)",Senna (2010),"Treasure of the Sierra Madre, The (1948)",Lawrence of Arabia (1962),L.A. Confidential (1997),Monty Python's Life of Brian (1979)
5,Vertigo (1958),Casablanca (1942),Up (2009),When We Were Kings (1996),Once Upon a Time in the West (C'era una volta ...,"Bridge on the River Kwai, The (1957)",Vertigo (1958),Annie Hall (1977)
6,Blade Runner (1982),To Kill a Mockingbird (1962),WALLÂ·E (2008),Touching the Void (2003),There Will Be Blood (2007),Schindler's List (1993),Reservoir Dogs (1992),One Flew Over the Cuckoo's Nest (1975)
7,Reservoir Dogs (1992),Fargo (1996),My Neighbor Totoro (Tonari no Totoro) (1988),Super Size Me (2004),For a Few Dollars More (Per qualche dollaro in...,Full Metal Jacket (1987),"Maltese Falcon, The (1941)",Monty Python and the Holy Grail (1975)
8,Fargo (1996),Apocalypse Now (1979),Toy Story (1995),Citizenfour (2014),High Noon (1952),M*A*S*H (a.k.a. MASH) (1970),"Third Man, The (1949)","Clockwork Orange, A (1971)"
9,Pulp Fiction (1994),Pulp Fiction (1994),Toy Story 3 (2010),"Thin Blue Line, The (1988)",True Grit (2010),Forrest Gump (1994),Memento (2000),Young Frankenstein (1974)


The next topic is real-time events. Personalize has the ability to listen to events from your application in order to update the recommendations shown to the user. This is especially useful in media workloads, like video-on-demand, where a customer's intent may differ based on if they are watching with their children or on their own.

Additionally the events that are recorded via this system are stored until a delete call from you is issued, and they are used as historical data alongside the other interaction data you provided when you train your next models.

#### Real time events

Start by creating an event tracker that is attached to the campaign.

In [108]:
response = personalize.create_event_tracker(
    name='MovieTracker',
    datasetGroupArn=dataset_group_arn
)
print(response['eventTrackerArn'])
print(response['trackingId'])
TRACKING_ID = response['trackingId']
event_tracker_arn = response['eventTrackerArn']

arn:aws:personalize:us-east-1:051545784337:event-tracker/5ec93b79
c2b6edca-7456-4634-93d8-4a60ffa460b5


We will create some code that simulates a user interacting with a particular item. After running this code, you will get recommendations that differ from the results above.

We start by creating some methods for the simulation of real time events.

In [109]:
session_dict = {}

def send_movie_click(USER_ID, ITEM_ID, EVENT_TYPE):
    """
    Simulates a click as an envent
    to send an event to Amazon Personalize's Event Tracker
    """
    # Configure Session
    try:
        session_ID = session_dict[str(USER_ID)]
    except:
        session_dict[str(USER_ID)] = str(uuid.uuid1())
        session_ID = session_dict[str(USER_ID)]
        
    # Configure Properties:
    event = {
    "itemId": str(ITEM_ID),
    }
    event_json = json.dumps(event)
        
    # Make Call
    
    personalize_events.put_events(
    trackingId = TRACKING_ID,
    userId= str(USER_ID),
    sessionId = session_ID,
    eventList = [{
        'sentAt': int(time.time()),
        'eventType': str(EVENT_TYPE),
        'properties': event_json
        }]
    )

def get_new_recommendations_df_users_real_time(recommendations_df, user_id, item_id, event_type):
    # Get the artist name (header of column)
    movie_name = get_movie_by_id(item_id)
    # Interact with different movies
    print('sending event ' + event_type + ' for ' + get_movie_by_id(item_id))
    send_movie_click(USER_ID=user_id, ITEM_ID=item_id, EVENT_TYPE=event_type)
    # Get the recommendations (note you should have a base recommendation DF created before)
    get_recommendations_response = personalize_runtime.get_recommendations(
        campaignArn = userpersonalization_campaign_arn,
        userId = str(user_id),
    )
    # Build a new dataframe of recommendations
    item_list = get_recommendations_response['itemList']
    recommendation_list = []
    for item in item_list:
        artist = get_movie_by_id(item['itemId'])
        recommendation_list.append(artist)
    new_rec_DF = pd.DataFrame(recommendation_list, columns = [movie_name])
    # Add this dataframe to the old one
    #recommendations_df = recommendations_df.join(new_rec_DF)
    recommendations_df = pd.concat([recommendations_df, new_rec_DF], axis=1)
    return recommendations_df

At this point, we haven't generated any real-time events yet; we have only set up the code. To compare the recommendations before and after the real-time events, let's pick one user and generate the original recommendations for them.

In [120]:
# First pick a user
user_id = user

# Get recommendations for the user
get_recommendations_response = personalize_runtime.get_recommendations(
        campaignArn = userpersonalization_campaign_arn,
        userId = str(user_id),
    )

# Build a new dataframe for the recommendations
item_list = get_recommendations_response['itemList']
recommendation_list = []
for item in item_list:
    artist = get_movie_by_id(item['itemId'])
    recommendation_list.append(artist)
user_recommendations_df = pd.DataFrame(recommendation_list, columns = [user_id])
user_recommendations_df

,43392
0,Predator (1987)
1,"Shining, The (1980)"
2,Close Encounters of the Third Kind (1977)
3,"Terminator, The (1984)"
4,"Thing, The (1982)"
5,Alien (1979)
6,2001: A Space Odyssey (1968)
7,Aliens (1986)
8,"Abyss, The (1989)"
9,"Exorcist, The (1973)"


Ok, so now we have a list of recommendations for this user before we have applied any real-time events. Now let's pick 3 random artists which we will simulate our user interacting with, and then see how this changes the recommendations.

In [111]:
# Next generate 3 random movies
movies = items_df.sample(3).index.tolist()

In [112]:
# Note this will take about 15 seconds to complete due to the sleeps
for movie in movies:
    time.sleep(5)
    user_recommendations_df = get_new_recommendations_df_users_real_time(user_recommendations_df, user_id, movie,'click')
user_recommendations_df

sending event click for Someone's Watching Me! (1978)
sending event click for The Mole Song: Undercover Agent Reiji (2014)
sending event click for Starship (1984)


,43392,Someone's Watching Me! (1978),The Mole Song: Undercover Agent Reiji (2014),Starship (1984)
0,Chinatown (1974),Chinatown (1974),Chinatown (1974),Psycho (1960)
1,"Sting, The (1973)","Sting, The (1973)","Sting, The (1973)","Graduate, The (1967)"
2,Citizen Kane (1941),Citizen Kane (1941),Citizen Kane (1941),Taxi Driver (1976)
3,Rear Window (1954),Rear Window (1954),Rear Window (1954),Dr. Strangelove or: How I Learned to Stop Worr...
4,North by Northwest (1959),North by Northwest (1959),North by Northwest (1959),"Sting, The (1973)"
5,"Graduate, The (1967)","Graduate, The (1967)","Graduate, The (1967)",Chinatown (1974)
6,Taxi Driver (1976),Taxi Driver (1976),Taxi Driver (1976),Vertigo (1958)
7,L.A. Confidential (1997),L.A. Confidential (1997),L.A. Confidential (1997),"Grand Budapest Hotel, The (2014)"
8,Goodfellas (1990),Goodfellas (1990),Goodfellas (1990),"Good, the Bad and the Ugly, The (Buono, il bru..."
9,Vertigo (1958),Vertigo (1958),Vertigo (1958),Run Lola Run (Lola rennt) (1998)


In the cell above, the first column after the index is the user's default recommendations from HRNN, and each column after that has a header of the artist that they interacted with via a real time event, and the recommendations after this event occurred. 

The behavior may not shift very much; this is due to the relatively limited nature of this dataset and effect of a few random clicks. If you wanted to better understand this, try simulating clicking more movies, and you should see a more pronounced impact.

Now lets look at the event filters, which allow you to filter items based on the interaction data. For this dataset, it could be click or watch based on the data we imported, but could be based on whatever interaction schema you design (click, rate, like, watch, purchase etc.) For VOD shelves you could move a title from "Top picks for you" to a "Watch again", the watch again recommendations will be based on the users current interactions, but only recommend titles that have already been watched.


In [113]:
recommendations_df_events = pd.DataFrame()
for filter_arn in interaction_filter_arns:
    recommendations_df_events = get_new_recommendations_df_by_filter(recommendations_df_events, user, filter_arn)
    
recommendations_df_events

,watched,unwatched
0,Alien (1979),North by Northwest (1959)
1,Pulp Fiction (1994),"Graduate, The (1967)"
2,Dr. Strangelove or: How I Learned to Stop Worr...,L.A. Confidential (1997)
3,Chinatown (1974),Vertigo (1958)
4,Goodfellas (1990),Blade Runner (1982)
5,Citizen Kane (1941),Psycho (1960)
6,Taxi Driver (1976),Reservoir Dogs (1992)
7,Fargo (1996),To Kill a Mockingbird (1962)
8,"Shawshank Redemption, The (1994)",Apocalypse Now (1979)
9,NaN,Monty Python's Life of Brian (1979)


now lets send a watch event in for 4 unwatched recommendations, which would simulate watching 4 movies. In a VOD application, you may choose to send in an event after they have watched a significant amount (over 75%) of a piece of content. Sending at 100% complete could miss people that stop short of the credits.

In [126]:
 # Get the recommendations
top_unwatched_recommendations_response = personalize_runtime.get_recommendations(
    campaignArn = userpersonalization_campaign_arn,
    userId = str(user_id),
    filterArn = filter_arn,
    numResults=4)
item_list = top_unwatched_recommendations_response['itemList']
for item in item_list:
    print('sending event watch for ' + get_movie_by_id(item['itemId']))
    send_movie_click(USER_ID=user_id, ITEM_ID=item['itemId'], EVENT_TYPE='watch')
    time.sleep(10)


sending event watch for Abyss, The (1989)
sending event watch for Exorcist, The (1973)
sending event watch for RoboCop (1987)
sending event watch for Planet of the Apes (1968)


Now we can look at the event filters to see the updated watched and unwatched recommendations 

In [125]:
recommendations_df_events = pd.DataFrame()
for filter_arn in interaction_filter_arns:
    recommendations_df_events = get_new_recommendations_df_by_filter(recommendations_df_events, user, filter_arn)
    
recommendations_df_events

,watched,unwatched
0,Predator (1987),"Abyss, The (1989)"
1,"Shining, The (1980)","Exorcist, The (1973)"
2,Close Encounters of the Third Kind (1977),RoboCop (1987)
3,"Terminator, The (1984)",Planet of the Apes (1968)
4,"Thing, The (1982)",E.T. the Extra-Terrestrial (1982)
5,Alien (1979),Mad Max (1979)
6,2001: A Space Odyssey (1968),Back to the Future Part II (1989)
7,Aliens (1986),Psycho (1960)
8,Ghostbusters (a.k.a. Ghost Busters) (1984),Total Recall (1990)
9,Blade Runner (1982),"Fifth Element, The (1997)"


### Personalized Ranking

The core use case for personalized ranking is to take a collection of items and to render them in priority or probable order of interest for a user. For a VOD application you want dynamically render a personalized shelf/rail/carousel based on some information (director, location, superhero franchise, movie time period etc). This may not be information that you have in your metadata, so a item metadata filter will not work, howeverr you may have this information within you system to generate the item list. 

To demonstrate this, we will use the same user from before and a random collection of items.

In [129]:
rerank_user = user
rerank_items = items_df.sample(25).index.tolist()

Now build a nice dataframe that shows the input data.

In [130]:
rerank_list = []
for item in rerank_items:
    movie = get_movie_by_id(item)
    rerank_list.append(movie)
rerank_df = pd.DataFrame(rerank_list, columns = ['Un-Ranked'])
rerank_df

,Un-Ranked
0,Political Animals (2016)
1,Take Me Home (2011)
2,Let's Not Keep in Touch! (1994)
3,Common (2014)
4,Kiki's Delivery Service (2014)
5,"Jewel of the Nile, The (1985)"
6,"Daddy, I'm A Zombie (2012)"
7,Moving Out (1983)
8,The Diabolical (2015)
9,"Live a Little, Love a Little (1968)"


Then make the personalized ranking API call.

In [131]:
# Convert user to string:
user_id = str(rerank_user)
rerank_item_list = []
for item in rerank_items:
    rerank_item_list.append(str(item))
    
# Get recommended reranking
get_recommendations_response_rerank = personalize_runtime.get_personalized_ranking(
        campaignArn = rerank_campaign_arn,
        userId = user_id,
        inputList = rerank_item_list
)

get_recommendations_response_rerank

{'ResponseMetadata': {'RequestId': 'dd264a82-740c-456c-ab5a-5dab64d8c890',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'content-type': 'application/json',
   'date': 'Fri, 11 Sep 2020 10:53:10 GMT',
   'x-amzn-requestid': 'dd264a82-740c-456c-ab5a-5dab64d8c890',
   'content-length': '1446',
   'connection': 'keep-alive'},
  'RetryAttempts': 0},
 'personalizedRanking': [{'itemId': '2405', 'score': 0.6703293},
  {'itemId': '48696', 'score': 0.2017058},
  {'itemId': '3937', 'score': 0.033551},
  {'itemId': '5196', 'score': 0.0320341},
  {'itemId': '371', 'score': 0.0282682},
  {'itemId': '102956', 'score': 0.0083296},
  {'itemId': '6391', 'score': 0.0064792},
  {'itemId': '117448', 'score': 0.004375},
  {'itemId': '41434', 'score': 0.0033204},
  {'itemId': '126577', 'score': 0.0025718},
  {'itemId': '180749', 'score': 0.0025118},
  {'itemId': '119754', 'score': 0.0019669},
  {'itemId': '144782', 'score': 0.0019532},
  {'itemId': '163428', 'score': 0.0016361},
  {'itemId': '151487', 'score':

Now add the reranked items as a second column to the original dataframe, for a side-by-side comparison.

In [132]:
ranked_list = []
item_list = get_recommendations_response_rerank['personalizedRanking']
for item in item_list:
    movie = get_movie_by_id(item['itemId'])
    ranked_list.append(movie)
ranked_df = pd.DataFrame(ranked_list, columns = ['Re-Ranked'])
rerank_df = pd.concat([rerank_df, ranked_df], axis=1)
rerank_df

,Un-Ranked,Re-Ranked
0,Political Animals (2016),"Jewel of the Nile, The (1985)"
1,Take Me Home (2011),Little Children (2006)
2,Let's Not Keep in Touch! (1994),Runaway (1984)
3,Common (2014),Lion of the Desert (1980)
4,Kiki's Delivery Service (2014),"Paper, The (1994)"
5,"Jewel of the Nile, The (1985)",Take Me Home (2011)
6,"Daddy, I'm A Zombie (2012)",Snowball Express (1972)
7,Moving Out (1983),Kiki's Delivery Service (2014)
8,The Diabolical (2015),After Innocence (2005)
9,"Live a Little, Love a Little (1968)","Daddy, I'm A Zombie (2012)"


You can see above how each entry was re-ordered based on the model's understanding of the user. This is a popular task when you have a collection of items to surface a user, a list of promotions for example.

## Batch recommendations <a class="anchor" id="batch"></a>
[Back to top](#top)

There are many cases where you may want to have a larger dataset of exported recommendations. Recently, Amazon Personalize launched batch recommendations as a way to export a collection of recommendations to S3. In this example, we will walk through how to do this for the HRNN solution. For more information about batch recommendations, please see the [documentation](https://docs.aws.amazon.com/personalize/latest/dg/getting-recommendations.html#recommendations-batch). This feature applies to all recipes, but the output format will vary.

A simple implementation looks like this:

```python
import boto3

personalize_rec = boto3.client(service_name='personalize')

personalize_rec.create_batch_inference_job (
    solutionVersionArn = "Solution version ARN",
    jobName = "Batch job name",
    roleArn = "IAM role ARN",
    jobInput = 
       {"s3DataSource": {"path": S3 input path}},
    jobOutput = 
       {"s3DataDestination": {"path":S3 output path"}}
)
```

The SDK import, the solution version arn, and role arns have all been determined. This just leaves an input, an output, and a job name to be defined.

Starting with the input for HRNN, it looks like:


```JSON
{"userId": "4638"}
{"userId": "663"}
{"userId": "3384"}
```

This should yield an output that looks like this:

```JSON
{"input":{"userId":"4638"}, "output": {"recommendedItems": ["296", "1", "260", "318"]}}
{"input":{"userId":"663"}, "output": {"recommendedItems": ["1393", "3793", "2701", "3826"]}}
{"input":{"userId":"3384"}, "output": {"recommendedItems": ["8368", "5989", "40815", "48780"]}}
```

The output is a JSON Lines file. It consists of individual JSON objects, one per line. So we will need to put in more work later to digest the results in this format.

### Building the input file

When you are using the batch feature, you specify the users that you'd like to receive recommendations for when the job has completed. The cell below will again select a few random users and will then build the file and save it to disk. From there, you will upload it to S3 to use in the API call later.

In [133]:
# We will use the same users from before
users
# Write the file to disk
json_input_filename = "json_input.json"
with open(data_dir + "/" + json_input_filename, 'w') as json_input:
    for user_id in users:
        json_input.write('{"userId": "' + str(user_id) + '"}\n')

In [134]:
# Showcase the input file:
!cat $data_dir"/"$json_input_filename

{"userId": "93562"}
{"userId": "61007"}
{"userId": "43392"}


Upload the file to S3 and save the path as a variable for later.

In [135]:
# Upload files to S3
boto3.Session().resource('s3').Bucket(bucket_name).Object(json_input_filename).upload_file(data_dir+"/"+json_input_filename)
s3_input_path = "s3://" + bucket_name + "/" + json_input_filename
print(s3_input_path)

s3://051545784337personalizepocvod/json_input.json


Batch recommendations read the input from the file we've uploaded to S3. Similarly, batch recommendations will save the output to file in S3. So we define the output path where the results should be saved.

In [136]:
# Define the output path
s3_output_path = "s3://" + bucket_name + "/"
print(s3_output_path)

s3://051545784337personalizepocvod/


Now just make the call to kick off the batch export process.

In [137]:
batchInferenceJobArn = personalize.create_batch_inference_job (
    solutionVersionArn = userpersonalization_solution_version_arn,
    jobName = "VOD-POC-Batch-Inference-Job-UserPersonalization_" + str(round(time.time()*1000)),
    roleArn = role_arn,
    jobInput = 
     {"s3DataSource": {"path": s3_input_path}},
    jobOutput = 
     {"s3DataDestination":{"path": s3_output_path}}
)
batchInferenceJobArn = batchInferenceJobArn['batchInferenceJobArn']

Run the while loop below to track the status of the batch recommendation call. This can take around 30 minutes to complete, because Personalize needs to stand up the infrastructure to perform the task. We are testing the feature with a dataset of only 3 users, which is not an efficient use of this mechanism. Normally, you would only use this feature for bulk processing, in which case the efficiencies will become clear.

In [138]:
current_time = datetime.now()
print("Import Started on: ", current_time.strftime("%I:%M:%S %p"))

max_time = time.time() + 6*60*60 # 6 hours
while time.time() < max_time:
    describe_dataset_inference_job_response = personalize.describe_batch_inference_job(
        batchInferenceJobArn = batchInferenceJobArn
    )
    status = describe_dataset_inference_job_response["batchInferenceJob"]['status']
    print("DatasetInferenceJob: {}".format(status))
    
    if status == "ACTIVE" or status == "CREATE FAILED":
        break
        
    time.sleep(60)
    
current_time = datetime.now()
print("Import Completed on: ", current_time.strftime("%I:%M:%S %p"))

Import Started on:  10:53:45 AM
DatasetInferenceJob: CREATE PENDING
DatasetInferenceJob: CREATE IN_PROGRESS
DatasetInferenceJob: CREATE IN_PROGRESS
DatasetInferenceJob: CREATE IN_PROGRESS
DatasetInferenceJob: CREATE IN_PROGRESS
DatasetInferenceJob: CREATE IN_PROGRESS
DatasetInferenceJob: CREATE IN_PROGRESS
DatasetInferenceJob: CREATE IN_PROGRESS
DatasetInferenceJob: CREATE IN_PROGRESS
DatasetInferenceJob: CREATE IN_PROGRESS
DatasetInferenceJob: CREATE IN_PROGRESS
DatasetInferenceJob: CREATE IN_PROGRESS
DatasetInferenceJob: CREATE IN_PROGRESS
DatasetInferenceJob: CREATE IN_PROGRESS
DatasetInferenceJob: CREATE IN_PROGRESS
DatasetInferenceJob: CREATE IN_PROGRESS
DatasetInferenceJob: CREATE IN_PROGRESS
DatasetInferenceJob: CREATE IN_PROGRESS
DatasetInferenceJob: CREATE IN_PROGRESS
DatasetInferenceJob: CREATE IN_PROGRESS
DatasetInferenceJob: CREATE IN_PROGRESS
DatasetInferenceJob: CREATE IN_PROGRESS
DatasetInferenceJob: CREATE IN_PROGRESS
DatasetInferenceJob: CREATE IN_PROGRESS
DatasetInfer

In [139]:
s3 = boto3.client('s3')
export_name = json_input_filename + ".out"
s3.download_file(bucket_name, export_name, data_dir+"/"+export_name)

# Update DF rendering
pd.set_option('display.max_rows', 30)
with open(data_dir+"/"+export_name) as json_file:
    # Get the first line and parse it
    line = json.loads(json_file.readline())
    # Do the same for the other lines
    while line:
        # extract the user ID 
        col_header = "User: " + line['input']['userId']
        # Create a list for all the artists
        recommendation_list = []
        # Add all the entries
        for item in line['output']['recommendedItems']:
            movie = get_movie_by_id(item)
            recommendation_list.append(movie)
        if 'bulk_recommendations_df' in locals():
            new_rec_DF = pd.DataFrame(recommendation_list, columns = [col_header])
            bulk_recommendations_df = bulk_recommendations_df.join(new_rec_DF)
        else:
            bulk_recommendations_df = pd.DataFrame(recommendation_list, columns=[col_header])
        try:
            line = json.loads(json_file.readline())
        except:
            line = None
bulk_recommendations_df

,User: 295,User: 73,User: 299,User: 93562,User: 61007,User: 43392
0,Raiders of the Lost Ark (Indiana Jones and the...,"Avengers, The (2012)",Ghostbusters (a.k.a. Ghost Busters) (1984),Big Fish (2003),Reservoir Dogs (1992),Chinatown (1974)
1,"Matrix, The (1999)",The Hunger Games (2012),Poltergeist (1982),Ice Age (2002),Office Space (1999),"Sting, The (1973)"
2,Saving Private Ryan (1998),Harry Potter and the Deathly Hallows: Part 2 (...,Jaws (1975),Ratatouille (2007),"Big Lebowski, The (1998)",Citizen Kane (1941)
3,Monty Python and the Holy Grail (1975),Man of Steel (2013),"Shining, The (1980)",Slumdog Millionaire (2008),Snatch (2000),Rear Window (1954)
4,Goldfinger (1964),Iron Man 2 (2010),"Silence of the Lambs, The (1991)",Harry Potter and the Order of the Phoenix (2007),Full Metal Jacket (1987),North by Northwest (1959)
5,Heat (1995),Captain America: The Winter Soldier (2014),Alien (1979),District 9 (2009),Trainspotting (1996),"Graduate, The (1967)"
6,Rear Window (1954),"Wolverine, The (2013)",Airplane! (1980),"Prestige, The (2006)",Monty Python's Life of Brian (1979),Taxi Driver (1976)
7,Pirates of the Caribbean: The Curse of the Bla...,Thor: The Dark World (2013),Blade Runner (1982),"Bourne Supremacy, The (2004)","Lock, Stock & Two Smoking Barrels (1998)",L.A. Confidential (1997)
8,For a Few Dollars More (Per qualche dollaro in...,The Hunger Games: Catching Fire (2013),"Exorcist, The (1973)",Shaun of the Dead (2004),"Clockwork Orange, A (1971)",Goodfellas (1990)
9,North by Northwest (1959),Guardians of the Galaxy (2014),"Terminator, The (1984)",Harry Potter and the Prisoner of Azkaban (2004),Hot Fuzz (2007),Vertigo (1958)


## Wrap up <a class="anchor" id="wrapup"></a>
[Back to top](#top)

With that you now have a fully working collection of models to tackle various recommendation and personalization scenarios, as well as the skills to manipulate customer data to better integrate with the service, and a knowledge of how to do all this over APIs and by leveraging open source data science tools.

Use these notebooks as a guide to getting started with your customers for POCs. As you find missing components, or discover new approaches, cut a pull request and provide any additional helpful components that may be missing from this collection.

You'll want to make sure that you clean up all of the resources deployed during this POC. We have provided a separate notebook which shows you how to identify and delete the resources in `04_Clean_Up_Resources.ipynb`.

In [140]:
%store event_tracker_arn
%store batchInferenceJobArn

Stored 'event_tracker_arn' (str)
Stored 'batchInferenceJobArn' (str)
